#### Notebook to preprocess dummy data

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, LineString
import random

In [ ]:
# Params
nsw_lambert_crs = "EPSG:28356"
target_distance = 10000  # in meters

#### Get data
- QGIS to create start and end point for transmission line
- QGIS to create project and NO-GO zones
- ELVIS to get DEM data

#### Reading up different RRT algorithms
- https://github.com/AtsushiSakai/PythonRobotics

### Attempting simple RRT with networkx graph

Algorithm:

1. Insert start point in the tree
2. While tree cannot connect to goal
    - Sample random point r in the image space
    - Find point p in the tree that is closest to r
    - Add branch of defined length from p to r
    - If new branch intersects with obstacle then either discard or trim it
3. Find the path from start to end through the tree

#### Helper functions
- Check intersection with exclusion zone
- Trim/discard branch if intersects
- Distance and angle from nearest node
- Find nearest node
- Randomly generate node in boundary

In [ ]:
def find_nearest_approved_to_ending(ending_point, approved_points):
    nearest_point = None
    min_distance = float('inf')
    for approved_point in approved_points:
        distance = ending_point.distance(approved_point)
        if distance < min_distance:
            min_distance = distance
            nearest_point = approved_point
    return nearest_point, min_distance

In [ ]:
def generate_target_circle(closest_approved_point, target_distance):
    return closest_approved_point.buffer(target_distance)

In [ ]:
def generage_rand_next_point(cur_point, target_circle, boundary_polygon, exclusion_zones, max_attempts=100):
    # Find intersection of target_circle and boundary polygon
    intersection_polygon = target_circle.intersection(boundary_polygon)
    # Get all coordinates on the boundary of this new polygon, and randomly pick one
    intersection_coords = list(intersection_polygon.exterior.coords)

    # Check if straight line between cur_point and attempt_point intersect with exclusion
    for attempt in range(max_attempts):
        attempt_point_coords = random.choice(intersection_coords)
        attempt_point = Point(attempt_point_coords)

        line_to_attempt = LineString([cur_point, attempt_point])

        intersects_exclusion = False
        for exclusion_zone in exclusion_zones.geometry:
            if line_to_attempt.intersects(exclusion_zone):
                intersects_exclusion = True
                break

        if not intersects_exclusion:
            return attempt_point
        
    # If max attempts exhusted return dead_end
    return "dead_end"

#### Initial attempt of RRT algorithm

In [ ]:
# Load starting point, boundary, exclusion zones
starting_point_gdf = gpd.read_file("starting_point.gpkg").to_crs(nsw_lambert_crs)
ending_point_gdf = gpd.read_file("ending_point.gpkg").to_crs(nsw_lambert_crs)
boundary_gdf = gpd.read_file("boundary.gpkg").to_crs(nsw_lambert_crs)
exclusion_zones_gdf = gpd.read_file("exclusion_zones.gpkg").to_crs(nsw_lambert_crs)

In [ ]:
# Create collection of approved points
approved_points = []
# Add starting point
approved_points.append(starting_point_gdf.geometry.iloc[0])

In [ ]:
# Find closest approved point to ending point
closest_approved_point, distance_to_closest = find_nearest_approved_to_ending(ending_point_gdf.geometry.iloc[0], approved_points)

In [ ]:
# Randomly generate a target point that is 'target_distance' away from any approved point within the boundary polygon
boundary_polygon = boundary_gdf.geometry.union_all()


#### Add in costs and weights